In [22]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import nltk
from bs4 import BeautifulSoup
import requests 
import re
import time
from datetime import date, datetime,time
import pymongo

In [2]:
india='https://news.google.com/topics/CAAqJQgKIh9DQkFTRVFvSUwyMHZNRE55YXpBU0JXVnVMVWRDS0FBUAE?hl=en-IN&gl=IN&ceid=IN%3Aen'
world='https://news.google.com/topics/CAAqKggKIiRDQkFTRlFvSUwyMHZNRGx1YlY4U0JXVnVMVWRDR2dKSlRpZ0FQAQ?hl=en-IN&gl=IN&ceid=IN%3Aen'
business='https://news.google.com/topics/CAAqKggKIiRDQkFTRlFvSUwyMHZNRGx6TVdZU0JXVnVMVWRDR2dKSlRpZ0FQAQ?hl=en-IN&gl=IN&ceid=IN%3Aen'
technology='https://news.google.com/topics/CAAqKggKIiRDQkFTRlFvSUwyMHZNRGRqTVhZU0JXVnVMVWRDR2dKSlRpZ0FQAQ?hl=en-IN&gl=IN&ceid=IN%3Aen'
entertainment='https://news.google.com/topics/CAAqKggKIiRDQkFTRlFvSUwyMHZNREpxYW5RU0JXVnVMVWRDR2dKSlRpZ0FQAQ?hl=en-IN&gl=IN&ceid=IN%3Aen'
sports='https://news.google.com/topics/CAAqKggKIiRDQkFTRlFvSUwyMHZNRFp1ZEdvU0JXVnVMVWRDR2dKSlRpZ0FQAQ?hl=en-IN&gl=IN&ceid=IN%3Aen'
science='https://news.google.com/topics/CAAqKggKIiRDQkFTRlFvSUwyMHZNRFp0Y1RjU0JXVnVMVWRDR2dKSlRpZ0FQAQ?hl=en-IN&gl=IN&ceid=IN%3Aen'
health='https://news.google.com/topics/CAAqJQgKIh9DQkFTRVFvSUwyMHZNR3QwTlRFU0JXVnVMVWRDS0FBUAE?hl=en-IN&gl=IN&ceid=IN%3Aen'

In [3]:
def fetch_respone(url,category):
    """
    The fuction take google news url as input and for several articles on the page it scrapes the article content and stores 
    as text files in respective folder

    """

    response = requests.get(url) ### using request module to take and store response of the url
    soup = BeautifulSoup(response.text, 'html.parser') ### parsing the response and souping out html text 

    """
    For each article scraping out the article content and appending to the list 

    """
    article_link_list=[]
    domain='https://news.google.com'
    for n in soup.find_all('article'):
        for x in n.find_all('a'): 
            article_link_list.append (domain+x.get('href')[1:])
    t=time.time()
    print(len(article_link_list))

    data_to_insert = {}
    data_to_insert['Category'] = category  # Assigning a category
    # List to store news details
    news_details = []
    
    # Iterate over each link in the list of article links
    for link in article_link_list:
        try:
            # Fetch the HTML content of the article link and create a BeautifulSoup object
            soup = BeautifulSoup(requests.get(link).text, 'html.parser')
            
            try:
                # Initialize an empty string to store the article text
                article_text = ''
                
                # Extract text from all <p> tags and append to the article_text
                for p_text in soup.find_all('p'):
                    article_text = article_text + p_text.text
                
                # Create a dictionary to store news details
                news = {
                    'title': soup.find_all('h1')[0].text,  # Extract title from the first <h1> tag
                    'article': article_text,  # Assign article text
                    'url': link,  # Assign article URL
                    'timestamp': datetime.today()  # Assign current date as timestamp
                }
                
                # Append news details to the list of news
                news_details.append(news)
            
            except Exception as e:
                # Handle exceptions if any
                pass
        
        except Exception as e:
            # Handle exceptions if any
            pass
    data_to_insert['news']=news_details
    print(time.time()-t) 
    return data_to_insert

In [4]:
final_collection=[]
a0=fetch_respone(india,'india')
a1=fetch_respone(world,'world')
a=fetch_respone(business,'business')
b=fetch_respone(technology,'technology')
c=fetch_respone(entertainment,'entertainment')
d=fetch_respone(sports,'sports')
e=fetch_respone(science,'science')
f=fetch_respone(health,'health')


545
733.9230215549469
523
727.6835308074951
534
711.3960950374603
496
635.517110824585
517
768.3167524337769
519
662.4009916782379
247
365.3033711910248
412
572.9857876300812


In [5]:
final_collection.append(a0)
final_collection.append(a1)
final_collection.append(a)
final_collection.append(b)
final_collection.append(c) 
final_collection.append(d)
final_collection.append(e)
final_collection.append(f)

In [3]:
# response = requests.get(india) ### using request module to take and store response of the url
# soup = BeautifulSoup(response.text, 'html.parser') ### parsing the response and souping out html text 
# article_link_list=[]
# domain='https://news.google.com'
# for n in soup.find_all('article'):
#     for x in n.find_all('a'): 
#         article_link_list.append (domain+x.get('href')[1:])
# t=time.time()

In [27]:
def wrtite_data_to_mongo(data):
    # Connect to MongoDB
    client = pymongo.MongoClient("mongodb://localhost:27017/")
    db = client["News_articles"]
    collection = db["daily_news"]    
    # Insert data into the collection
    collection.insert_many(data)
    
    # Print success message
    print("Data inserted successfully!")
wrtite_data_to_mongo(final_collection)

Data inserted successfully!
